Ноутбук запускался в колабе

In [1]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if device == torch.device('cpu'):
    print('Using cpu')
else:
    n_gpu = torch.cuda.device_count()
    print('Using {} GPUs'.format(torch.cuda.get_device_name(0)))

Using Tesla T4 GPUs


In [2]:
! pip install datasets transformers seqeval

     |████████████████████████████████| 194kB 17.1MB/s 
     |████████████████████████████████| 2.1MB 39.3MB/s 
     |████████████████████████████████| 51kB 8.8MB/s 
     |████████████████████████████████| 245kB 49.9MB/s 
     |████████████████████████████████| 112kB 59.0MB/s 
     |████████████████████████████████| 870kB 56.7MB/s 
     |████████████████████████████████| 3.3MB 54.5MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-cp37-none-any.whl size=16172 sha256=5b6624000f9f6182f59e5abc1710862f8ceae64c75b65dce1166cb2b361891ff
  Stored in directory: /root/.cache/pip/wheels/52/df/1b/45d75646c37428f7e626214704a0e35bd3cfc32eda37e59e5f
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=eaf3dd6206f5564b4dddcc97f23af6e2b7a2e6b602b6135f8874612858bbce73
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built seqeval sacremoses


In [3]:
task = "ner"
batch_size = 8

# Data - PAD

In [4]:
!gdown --id 1cCXjHX9FAgouF0DWuWPO5qyBHVZDkXTQ

Downloading...
From: https://drive.google.com/uc?id=1cCXjHX9FAgouF0DWuWPO5qyBHVZDkXTQ
To: /content/NER_PAD_agg_test.csv
2.92MB [00:00, 46.3MB/s]


In [5]:
!gdown --id 1RFNBRcly96omdpNtYlEK6O1EuQs3ux7t

Downloading...
From: https://drive.google.com/uc?id=1RFNBRcly96omdpNtYlEK6O1EuQs3ux7t
To: /content/NER_PAD_agg_train.csv
11.7MB [00:00, 44.2MB/s]


In [6]:
!gdown --id 1AnLqXtSyJNBK7YmwuW6L6z2OYhE4RPTV

Downloading...
From: https://drive.google.com/uc?id=1AnLqXtSyJNBK7YmwuW6L6z2OYhE4RPTV
To: /content/NER_PAD_agg.csv
14.6MB [00:00, 55.2MB/s]


In [7]:
import pandas as pd
data = pd.read_csv('NER_PAD_agg.csv', converters={'tokens': eval, 'ner_tags': eval})

In [8]:
all_tags = set()
for _, row in data.iterrows():
    for tag in row.ner_tags:
        all_tags.add(tag)
all_tags = sorted(list(all_tags))

In [9]:
tag_to_idx = {t: i for i, t in enumerate(all_tags)}

In [10]:
label_list = {i: t for i, t in enumerate(all_tags)}

In [11]:
train = pd.read_csv('NER_PAD_agg_train.csv', converters={'tokens': eval, 'ner_tags': eval})
test = pd.read_csv('NER_PAD_agg_test.csv', converters={'tokens': eval, 'ner_tags': eval})

In [12]:
train.ner_tags = train.ner_tags.apply(lambda x : [tag_to_idx[label] for label in x])
test.ner_tags = test.ner_tags.apply(lambda x : [tag_to_idx[label] for label in x])

In [13]:
train.to_pickle('train.pkl')
test.to_pickle('test.pkl')

In [14]:
from datasets import load_dataset, load_metric

In [15]:
datasets = load_dataset('pandas', data_files={'train' : 'train.pkl', 'test' : 'test.pkl'})

Using custom data configuration default-dfe6c5ee389858c3


Dataset pandas downloaded and prepared to /root/.cache/huggingface/datasets/pandas/default-dfe6c5ee389858c3/0.0.0/3547ff13f14ebf30dc8f11ab416dcf7721c18de2eb7e054bf9c80b88e765a791. Subsequent calls will reuse this data.


In [16]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 3200
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 800
    })
})

# Tokenizer

In [20]:
from transformers import AutoTokenizer
model_checkpoint = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [21]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [22]:
label_all_tokens = False

In [23]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [24]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

Token indices sequence length is longer than the specified maximum sequence length for this model (954 > 512). Running this sequence through the model will result in indexing errors


Warning : есть последовательности больше, чем максимально возможная в BERT - 512

In [25]:
pd.Series(tokenized_datasets["train"]['input_ids']).apply(lambda x : len(x)).describe()

count    3200.000000
mean      454.245000
std       283.854426
min         6.000000
25%       207.000000
50%       405.000000
75%       566.000000
max      4271.000000
dtype: float64

In [26]:
pd.Series(tokenized_datasets["test"]['input_ids']).apply(lambda x : len(x)).describe()

count     800.000000
mean      453.612500
std       281.724152
min        47.000000
25%       202.000000
50%       406.000000
75%       566.500000
max      1796.000000
dtype: float64

In [27]:
train_tokenized = tokenized_datasets["train"].to_pandas()
test_tokenized = tokenized_datasets["test"].to_pandas()

In [28]:
train_tokenized.head(1)

,attention_mask,id,input_ids,labels,ner_tags,token_type_ids,tokens
0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",SV990115834_ОРГ__INM_16'04'09_3.pdf,"[101, 180, 37410, 10519, 125, 118, 190, 94389,...","[-100, 15, 15, -100, 15, -100, 15, 15, -100, -...","[15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[l, испр, 4-, v, moscow, quarterly, report, li..."


In [29]:
test_tokenized.head(1)

,attention_mask,id,input_ids,labels,ner_tags,token_type_ids,tokens
0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",B31005082_ОРГ__INM_15'39'47_1.pdf,"[101, 180, 559, 106, 62213, 1779, 122, 80765, ...","[-100, 15, 15, 15, 15, 15, 15, 15, -100, -100,...","[15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[l, т, !, приложение, №, 1, постановлению, пра..."


In [30]:
CHUNCK_SIZE = 512

def lists_to_chuncks(row):
    n = CHUNCK_SIZE - 2
    new_row = row.copy()
    for key in ['attention_mask', 'input_ids', 'labels', 'token_type_ids']:
        FIRST = new_row[key][0]
        LAST = new_row[key][-1]
        new_row[key] = new_row[key][1:-1]
        new_row[key] = [[FIRST] + list(new_row[key][i:i + n]) + [LAST] for i in range(0, len(new_row[key]), n)]
    return new_row

In [31]:
train_tokenized = train_tokenized.drop(columns = ['ner_tags', 'tokens']).apply(lists_to_chuncks, axis=1)
train_tokenized = train_tokenized.set_index(['id']).apply(pd.Series.explode).reset_index()

In [32]:
test_tokenized = test_tokenized.drop(columns = ['ner_tags', 'tokens']).apply(lists_to_chuncks, axis=1)
test_tokenized = test_tokenized.set_index(['id']).apply(pd.Series.explode).reset_index()

In [33]:
train_tokenized.to_pickle('train_tokenized.pkl')
test_tokenized.to_pickle('test_tokenized.pkl')

In [34]:
tokenized_datasets = load_dataset('pandas', data_files={'train' : 'train_tokenized.pkl', 'test' : 'test_tokenized.pkl'})

Using custom data configuration default-31029acdc8580177


Dataset pandas downloaded and prepared to /root/.cache/huggingface/datasets/pandas/default-31029acdc8580177/0.0.0/3547ff13f14ebf30dc8f11ab416dcf7721c18de2eb7e054bf9c80b88e765a791. Subsequent calls will reuse this data.


In [35]:
pd.Series(tokenized_datasets["train"]['input_ids']).apply(lambda x : len(x)).describe()

count    4415.000000
mean      329.787995
std       162.203175
min         3.000000
25%       184.000000
50%       358.000000
75%       512.000000
max       512.000000
dtype: float64

In [36]:
pd.Series(tokenized_datasets["test"]['input_ids']).apply(lambda x : len(x)).describe()

count    1090.000000
mean      333.458716
std       159.904559
min         3.000000
25%       183.000000
50%       369.500000
75%       512.000000
max       512.000000
dtype: float64

Теперь все последовательности нужной длины

# Model

In [37]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at 

Warning : модель надо обучить на наших данных

In [38]:
args = TrainingArguments(
    f"test-{task}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
)

In [39]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [40]:
metric = load_metric("seqeval")

In [42]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [43]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [44]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy,Runtime,Samples Per Second
1,0.123400,0.049985,0.777584,0.816659,0.796642,0.984946,44.209100,24.656000
2,0.045600,0.040994,0.816129,0.870812,0.842584,0.987706,43.851400,24.857000
3,0.033900,0.038767,0.823517,0.875860,0.848882,0.988467,44.000000,24.773000


TrainOutput(global_step=1656, training_loss=0.0641417177978921, metrics={'train_runtime': 1671.3849, 'train_samples_per_second': 0.991, 'total_flos': 7143003009383424.0, 'epoch': 3.0, 'init_mem_cpu_alloc_delta': 2022924288, 'init_mem_gpu_alloc_delta': 709382656, 'init_mem_cpu_peaked_delta': 0, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': -70258688, 'train_mem_gpu_alloc_delta': 2197179392, 'train_mem_cpu_peaked_delta': 489480192, 'train_mem_gpu_peaked_delta': 6504072192})

In [45]:
trainer.evaluate()

{'epoch': 3.0,
 'eval_accuracy': 0.9884670199665149,
 'eval_f1': 0.8488824641387747,
 'eval_loss': 0.03876703232526779,
 'eval_mem_cpu_alloc_delta': 3280896,
 'eval_mem_cpu_peaked_delta': 1474560,
 'eval_mem_gpu_alloc_delta': 0,
 'eval_mem_gpu_peaked_delta': 417779712,
 'eval_precision': 0.8235167206040992,
 'eval_recall': 0.8758604864616797,
 'eval_runtime': 40.6181,
 'eval_samples_per_second': 26.835}

In [46]:
predictions, labels, _ = trainer.predict(tokenized_datasets["test"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

{'DOCAGRDATE': {'f1': 0.7386363636363636,
  'number': 151,
  'precision': 0.6467661691542289,
  'recall': 0.8609271523178808},
 'DOCAGRNUM': {'f1': 0.6573033707865169,
  'number': 179,
  'precision': 0.6610169491525424,
  'recall': 0.6536312849162011},
 'DOCAMOUNT': {'f1': 0.8704061895551257,
  'number': 1244,
  'precision': 0.8389261744966443,
  'recall': 0.9043408360128617},
 'DOCCPTY': {'f1': 0.8408916218293621,
  'number': 596,
  'precision': 0.775886524822695,
  'recall': 0.9177852348993288},
 'DOCCPTYINN': {'f1': 0.9401709401709402,
  'number': 282,
  'precision': 0.9075907590759076,
  'recall': 0.975177304964539},
 'DOCCUSTOMER': {'f1': 0.9335443037974683,
  'number': 610,
  'precision': 0.9021406727828746,
  'recall': 0.9672131147540983},
 'DOCDATE': {'f1': 0.87527352297593,
  'number': 645,
  'precision': 0.8264462809917356,
  'recall': 0.9302325581395349},
 'DOCNUM': {'f1': 0.734520780322307,
  'number': 651,
  'precision': 0.8200757575757576,
  'recall': 0.6651305683563749},

In [48]:
pd.DataFrame.from_dict(results).transpose().iloc[:-4, :-1]

,precision,recall,f1
DOCAGRDATE,0.646766,0.860927,0.738636
DOCAGRNUM,0.661017,0.653631,0.657303
DOCAMOUNT,0.838926,0.904341,0.870406
DOCCPTY,0.775887,0.917785,0.840892
DOCCPTYINN,0.907591,0.975177,0.940171
DOCCUSTOMER,0.902141,0.967213,0.933544
DOCDATE,0.826446,0.930233,0.875274
DOCNUM,0.820076,0.665131,0.734521


In [49]:
pd.DataFrame.from_dict(results).transpose().iloc[-4:, 0].rename('result')

overall_precision    0.823517
overall_recall       0.875860
overall_f1           0.848882
overall_accuracy     0.988467
Name: result, dtype: float64

In [51]:
!pip install sklearn_crfsuite

     |████████████████████████████████| 747kB 18.6MB/s 


In [52]:
from sklearn_crfsuite import metrics
print(metrics.flat_classification_report(true_labels, true_predictions, labels=all_tags, digits=3))

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


               precision    recall  f1-score   support

 B-DOCAGRDATE      0.691     0.893     0.779       150
  B-DOCAGRNUM      0.689     0.682     0.685       179
  B-DOCAMOUNT      0.856     0.912     0.883      1243
    B-DOCCPTY      0.828     0.919     0.871       595
 B-DOCCPTYINN      0.908     0.975     0.940       282
B-DOCCUSTOMER      0.926     0.970     0.948       609
    B-DOCDATE      0.867     0.949     0.906       645
     B-DOCNUM      0.820     0.665     0.735       651
 I-DOCAGRDATE      0.650     0.864     0.741        88
  I-DOCAGRNUM      0.000     0.000     0.000         7
  I-DOCAMOUNT      0.829     0.881     0.854       620
    I-DOCCPTY      0.807     0.947     0.871       564
I-DOCCUSTOMER      0.896     0.955     0.925       462
    I-DOCDATE      0.896     0.952     0.923       605
     I-DOCNUM      0.000     0.000     0.000         0
            O      0.996     0.993     0.994    137842

    micro avg      0.988     0.988     0.988    144542
    macr